In [11]:
import pandas as pd
import numpy as np

In [14]:
dislok = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
dislok = dislok.drop(['plan_date', 'kod_vrab', 'date_kap', 'date_dep', 'id_road_disl', 'st_id_dest', 'id_road_dest', 'st_id_send', 'id_road_send', 'last_fr_id', 'ost_prob'], axis=1)
# dislok = dislok.groupby('wagnum').agg({'date_kap': 'max', 'date_dep': 'max', 'ost_prob': 'min', 'isload': 'mean', 'distance': 'sum'})
print(len(dislok))
dislok.head()

6250933


,wagnum,date_pl_rem,isload,fr_id,distance
0,1071,2023-02-16,0,5032,-1
1,1071,2023-02-16,1,1204,998
2,1071,2023-02-16,1,1204,989
3,1071,2023-02-16,1,1204,939
4,1071,2023-02-16,1,1204,963


In [15]:
dislok.isna().sum()

wagnum         0
date_pl_rem    0
isload         0
fr_id          0
distance       0
dtype: int64

In [16]:
freights = pd.read_parquet('freight_info.parquet').convert_dtypes()
freights = pd.get_dummies(freights, columns=['fr_class']).astype(int)
print(len(freights))
freights.head()

5079


,fr_id,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,4989,0,0,0,0,0,0,0,0,0,1,0
1,4990,0,0,0,0,0,0,0,0,0,1,0
2,4991,0,0,0,0,0,0,0,1,0,0,0
3,4992,0,0,0,0,0,0,0,0,0,1,0
4,4993,0,0,0,0,0,0,0,0,0,1,0


In [17]:
merged = pd.merge(dislok, freights, on='fr_id', how='left')
print(len(merged))
merged.head()

6250933


,wagnum,date_pl_rem,isload,fr_id,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,2023-02-16,0,5032,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,2023-02-16,1,1204,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,2023-02-16,1,1204,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,2023-02-16,1,1204,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,2023-02-16,1,1204,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
import sys
sys.path.append('..')
from src.timeconverter import date_to_timestamp

# merged['date_kap'] = merged['date_kap'].fillna('1970-01-01')
merged['date_pl_rem'] = merged['date_pl_rem'].astype(str).apply(date_to_timestamp)
# merged['date_dep'] = merged['date_dep'].fillna('1970-01-01')
# merged['date_dep'] = merged['date_dep'].astype(str).apply(date_to_timestamp)
merged.head()

,wagnum,date_pl_rem,isload,fr_id,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,1676505600,0,5032,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,1676505600,1,1204,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,1676505600,1,1204,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,1676505600,1,1204,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,1676505600,1,1204,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
merged.isna().sum()

wagnum              0
date_pl_rem         0
isload              0
fr_id               0
distance            0
skoroport      155015
naval          155015
nasip          155015
naliv          155015
openvagons     155015
soprovod       155015
smerz          155015
fr_class_0     155015
fr_class_1     155015
fr_class_2     155015
fr_class_3     155015
dtype: int64

In [20]:
# merged.loc[merged['date_kap'] == 0, 'date_kap'] = int(merged.loc[merged['date_kap'] != 0, 'date_kap'].mean())
# merged.loc[merged['date_dep'] == 0, 'date_dep'] = int(merged.loc[merged['date_dep'] != 0, 'date_dep'].mean())
# merged['ost_prob'] = merged['ost_prob'].fillna(int(merged['ost_prob'].mean()))
# merged.head()

In [21]:
missings = ['skoroport', 'naval', 'nasip', 'naliv', 'openvagons', 'soprovod', 'smerz', 'fr_class_0', 'fr_class_1', 'fr_class_2', 'fr_class_3']
merged[missings] = merged[missings].fillna(-1)
# merged.loc[merged['isload'] == 0, missings] = -1
merged = merged.drop('fr_id', axis=1)
print(len(merged))
merged.head()

6250933


,wagnum,date_pl_rem,isload,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,1676505600,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,1676505600,1,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,1676505600,1,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,1676505600,1,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,1676505600,1,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [23]:
grouped = merged.groupby('wagnum').agg({'date_pl_rem': 'max',
                                       'isload': 'mean', 
                                       'distance': 'sum',
                                       'skoroport': 'mean',
                                       'naval': 'mean',
                                       'nasip': 'mean',
                                       'naliv': 'mean',
                                       'openvagons': 'mean',
                                       'soprovod': 'mean',
                                       'smerz': 'mean',
                                       'fr_class_0': 'mean',
                                       'fr_class_1': 'mean',
                                       'fr_class_2': 'mean',
                                       'fr_class_3': 'mean',
                                    #    'date_kap': 'max', 
                                    #    'date_dep': 'max', 
                                    #    'ost_prob': 'min', 
                                    })
print(len(grouped))
grouped.head()

33977


,date_pl_rem,isload,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
wagnum,,,,,,,,,,,,,,
0,1683331200,0.347826,119051,0.000000,0.173913,0.173913,0.000000,0.000000,0.000000,0.000000,0.173913,0.706522,0.119565,0.000000
1,1719792000,0.407609,183024,-0.021739,-0.021739,-0.021739,-0.021739,-0.021739,-0.021739,-0.021739,0.233696,0.527174,0.152174,-0.021739
2,1724025600,0.288043,158751,-0.059783,0.086957,0.086957,-0.059783,-0.059783,-0.059783,-0.059783,0.086957,0.701087,-0.027174,-0.059783
3,1711929600,0.402174,122918,0.000000,0.081522,0.081522,0.000000,0.000000,0.000000,0.000000,0.108696,0.663043,0.228261,0.000000
4,1732752000,0.559783,12260,-0.081522,0.108696,0.108696,-0.081522,-0.081522,-0.081522,-0.081522,0.043478,0.614130,0.016304,-0.081522


In [24]:
grouped.to_csv('../preprocessed/dislok_and_freight.csv')